# Assemble Data

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.ndimage import uniform_filter1d  # simple smoothing
import re

/home/simonhans/anaconda3/envs/GrapeExpectations/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_rows', None)  # show all columns
# import os
# os.chdir('..')

### Start with plot features

In [3]:
# get elevation features
df = pd.read_pickle('../data/plot_elev_features.pkl')

In [4]:
df

,plot_id,slope_mean,slope_min,slope_max,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,aspect_mean,aspect_min,aspect_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha
0,0,5.020058,3.925700,6.804783,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,175.379315,155.373535,190.393188,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114
1,1,5.289605,3.356515,8.816703,-0.000258,-0.014769,0.013737,-0.001123,-0.016449,0.012478,-0.000258,-0.014769,0.013737,169.123825,144.559204,188.072983,206.874710,208.469513,207.776839,21.910873,23.505676,22.812984,"POLYGON ((-119.78179 45.87245, -119.78167 45.8...",1.594803,381.909845,0.038191
2,2,7.150476,4.206255,11.814886,0.000065,-0.030229,0.034444,-0.000754,-0.043223,0.027175,0.000065,-0.030229,0.034444,152.046614,135.318848,175.093628,205.507690,207.790955,206.759655,20.543854,22.827118,21.795817,"POLYGON ((-119.78139 45.87246, -119.78129 45.8...",2.283264,304.757142,0.030476
3,3,9.178217,5.255136,12.657069,0.000241,-0.022650,0.018867,0.000610,-0.020701,0.012653,0.000241,-0.022650,0.018867,175.202423,159.699402,189.218704,210.499161,214.308624,212.434447,25.535324,29.344788,27.470613,"POLYGON ((-119.78299 45.87287, -119.78300 45.8...",3.809464,416.261511,0.041626
4,4,6.641992,4.568419,10.110537,0.000753,-0.013419,0.014943,0.000611,-0.011132,0.012551,0.000753,-0.013419,0.014943,173.372281,156.050049,189.273071,209.319901,213.504730,211.261849,24.356064,28.540894,26.298020,"POLYGON ((-119.78263 45.87269, -119.78269 45.8...",4.184830,934.859055,0.093486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,3593,6.524146,4.537724,8.501439,-0.001274,-0.024254,0.019990,-0.001476,-0.019637,0.020838,-0.001274,-0.024254,0.019990,163.878749,140.306885,186.459793,228.098480,232.209412,230.327266,43.134644,47.245575,45.363431,"POLYGON ((-119.77610 45.87519, -119.77626 45.8...",4.110931,544.855651,0.054486
3594,3594,3.070807,1.769737,4.388859,0.000577,-0.013051,0.017839,0.000672,-0.015131,0.010445,0.000577,-0.013051,0.017839,176.192068,142.778198,228.122147,234.638351,235.519440,235.030851,49.674515,50.555603,50.067012,"POLYGON ((-119.77718 45.87568, -119.77718 45.8...",0.881088,319.126138,0.031913
3595,3595,3.290681,1.204210,6.403341,-0.001655,-0.029511,0.026497,-0.002227,-0.026850,0.022847,-0.001655,-0.029511,0.026497,210.650471,141.063812,265.084839,235.022568,236.736877,235.759766,50.058731,51.773041,50.795942,"POLYGON ((-119.77685 45.87573, -119.77680 45.8...",1.714310,591.653344,0.059165
3596,3596,6.540738,1.204210,9.352532,-0.000439,-0.029511,0.027310,-0.001295,-0.025244,0.019796,-0.000439,-0.029511,0.027310,140.301358,110.309143,226.955688,232.924850,236.760712,235.060714,47.961014,51.796875,50.096894,"POLYGON ((-119.77652 45.87579, -119.77642 45.8...",3.835861,847.012539,0.084701


### Compute vectors for aspect and slope, and some interaction terms

In [5]:
# Aspect: convert to radians and compute sin/cos
df['aspect_min_cos'] = np.cos(np.radians(df['aspect_min']))
df['aspect_min_sin'] = np.sin(np.radians(df['aspect_min']))

df['aspect_max_cos'] = np.cos(np.radians(df['aspect_max']))
df['aspect_max_sin'] = np.sin(np.radians(df['aspect_max']))

df['aspect_mean_cos'] = np.cos(np.radians(df['aspect_mean']))
df['aspect_mean_sin'] = np.sin(np.radians(df['aspect_mean']))

# Drop raw aspect values
df = df.drop(columns=['aspect_min', 'aspect_max', 'aspect_mean'])

df['slope_rad'] = np.radians(df['slope_mean'])
df['slope_grad'] = np.tan(df['slope_rad'])


df['slope_x'] = df['slope_grad'] * df['aspect_mean_cos']
df['slope_y'] = df['slope_grad'] * df['aspect_mean_sin']

df = df.drop(columns = ['slope_mean','slope_min','slope_max'])

In [6]:
df

,plot_id,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_rad,slope_grad,slope_x,slope_y
0,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076
1,1,-0.000258,-0.014769,0.013737,-0.001123,-0.016449,0.012478,-0.000258,-0.014769,0.013737,206.874710,208.469513,207.776839,21.910873,23.505676,22.812984,"POLYGON ((-119.78179 45.87245, -119.78167 45.8...",1.594803,381.909845,0.038191,-0.814715,0.579861,-0.990090,-0.140434,-0.982037,0.188687,0.092321,0.092584,-0.090921,0.017469
2,2,0.000065,-0.030229,0.034444,-0.000754,-0.043223,0.027175,0.000065,-0.030229,0.034444,205.507690,207.790955,206.759655,20.543854,22.827118,21.795817,"POLYGON ((-119.78139 45.87246, -119.78129 45.8...",2.283264,304.757142,0.030476,-0.711031,0.703161,-0.996336,0.085528,-0.883329,0.468753,0.124799,0.125451,-0.110815,0.058806
3,3,0.000241,-0.022650,0.018867,0.000610,-0.020701,0.012653,0.000241,-0.022650,0.018867,210.499161,214.308624,212.434447,25.535324,29.344788,27.470613,"POLYGON ((-119.78299 45.87287, -119.78300 45.8...",3.809464,416.261511,0.041626,-0.937885,0.346945,-0.987084,-0.160203,-0.996496,0.083636,0.160190,0.161575,-0.161008,0.013513
4,4,0.000753,-0.013419,0.014943,0.000611,-0.011132,0.012551,0.000753,-0.013419,0.014943,209.319901,213.504730,211.261849,24.356064,28.540894,26.298020,"POLYGON ((-119.78263 45.87269, -119.78269 45.8...",4.184830,934.859055,0.093486,-0.913900,0.405938,-0.986932,-0.161140,-0.993317,0.115418,0.115925,0.116447,-0.115669,0.013440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,3593,-0.001274,-0.024254,0.019990,-0.001476,-0.019637,0.020838,-0.001274,-0.024254,0.019990,228.098480,232.209412,230.327266,43.134644,47.245575,45.363431,"POLYGON ((-119.77610 45.87519, -119.77626 45.8...",4.110931,544.855651,0.054486,-0.769476,0.638675,-0.993651,-0.112506,-0.960676,0.277671,0.113868,0.114363,-0.109865,0.031755
3594,3594,0.000577,-0.013051,0.017839,0.000672,-0.015131,0.010445,0.000577,-0.013051,0.017839,234.638351,235.519440,235.030851,49.674515,50.555603,50.067012,"POLYGON ((-119.77718 45.87568, -119.77718 45.8...",0.881088,319.126138,0.031913,-0.796300,0.604902,-0.667545,-0.744570,-0.997792,0.066412,0.053596,0.053647,-0.053529,0.003563
3595,3595,-0.001655,-0.029511,0.026497,-0.002227,-0.026850,0.022847,-0.001655,-0.029511,0.026497,235.022568,236.736877,235.759766,50.058731,51.773041,50.795942,"POLYGON ((-119.77685 45.87573, -119.77680 45.8...",1.714310,591.653344,0.059165,-0.777846,0.628454,-0.085681,-0.996323,-0.860293,-0.509799,0.057433,0.057496,-0.049464,-0.029312
3596,3596,-0.000439,-0.029511,0.027310,-0.001295,-0.025244,0.019796,-0.000439,-0.029511,0.027310,232.924850,236.760712,235.060714,47.961014,51.796875,50.096894,"POLYGON ((-119.77652 45.87579, -119.77642 45.8...",3.835861,847.012539,0.084701,-0.347085,0.937834,-0.682564,-0.730826,-0.769415,0.638750,0.114157,0.114656,-0.088218,0.073236


## Now add NDVI for each plot to features

### open up the filtered and smoothed ndvi df

In [7]:
veg_agg = pd.read_pickle('../data/ndvi/plots/final_df.pkl')
keep_cols = ['plot_id', 'year']

# mean_cols = [col for col in veg_agg.columns 
#              if '_mean_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
# keep_cols.extend(mean_cols)

# slope_cols = [col for col in veg_agg.columns 
#               if '_slope_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
# keep_cols.extend(slope_cols)

# std_cols = [col for col in veg_agg.columns 
#             if '_std_' in col and re.match(r'.*_(\d+)$', col) and int(col.split('_')[-1]) <= 10]
# keep_cols.extend(std_cols)

# sos_cols = [col for col in veg_agg.columns if '_sos_month' in col]
# keep_cols.extend(sos_cols)

# season_length_cols = [col for col in veg_agg.columns if '_season_length' in col]
# keep_cols.extend(season_length_cols)

# senescence_cols = [col for col in veg_agg.columns if '_senescence_rate' in col]
# keep_cols.extend(senescence_cols)

# veg_agg = veg_agg[keep_cols].copy()

In [8]:
veg_agg = veg_agg.dropna(axis = 1)
veg_agg

,plot_id,year,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_cov,ndvi_mean,ndvi_std
0,0.0,2016,0.383043,0.373238,0.356197,0.348879,0.346145,0.345504,0.355702,0.381068,0.413351,0.438675,0.443236,0.440521,0.432348,0.424176,0.416003,0.407830,-0.001783,-0.001783,-0.001851,-0.000534,-0.000351,0.000406,0.003347,0.003689,0.005842,0.000652,0.000652,-0.001168,-0.001168,-0.001168,-0.001168,-0.001168,0.001993,0.003565,0.003942,0.001107,0.000702,0.000845,0.006752,0.007378,0.011685,0.001303,0.001303,0.002335,0.002335,0.002335,0.002335,0.002335,0.090644,0.393639,0.035681
1,0.0,2017,0.329507,0.322669,0.320248,0.318124,0.322323,0.324225,0.321754,0.322187,0.327781,0.330168,0.331366,0.333434,0.339194,0.339164,0.338132,0.337247,-0.001214,-0.000664,-0.000828,0.000356,0.000338,0.000090,-0.000614,0.001560,0.000333,0.000323,0.000109,0.000761,0.000437,-0.000147,-0.000147,-0.000147,0.000991,0.002033,0.001665,0.000953,0.000702,0.000415,0.001264,0.003149,0.000665,0.000660,0.000218,0.001577,0.001062,0.000295,0.000295,0.000209,0.022024,0.328595,0.007237
2,0.0,2018,0.313716,0.306276,0.305823,0.303524,0.306111,0.314969,0.329831,0.339823,0.344996,0.351547,0.354465,0.366220,0.378805,0.389178,0.428417,0.467546,-0.000188,-0.001330,-0.000526,0.000361,0.000039,0.002017,0.002040,0.000333,0.001936,0.000417,0.000417,0.003317,0.001256,0.002308,0.008811,0.001145,0.000466,0.003144,0.001261,0.000954,0.000679,0.004080,0.004080,0.000839,0.004117,0.000834,0.000834,0.006976,0.002513,0.004907,0.017655,0.002290,0.170850,0.365089,0.062376
3,0.0,2019,0.298241,0.300882,0.299702,0.299082,0.300565,0.304063,0.299246,0.293215,0.291676,0.290423,0.289507,0.288646,0.286824,0.277969,0.272042,0.266342,0.000334,0.000545,-0.000654,0.000671,0.000357,-0.000029,-0.000308,-0.000707,-0.000187,-0.000155,-0.000123,-0.000123,-0.000642,-0.001236,-0.000721,-0.000994,0.000273,0.001094,0.001499,0.001836,0.000820,0.002348,0.000633,0.001991,0.000374,0.000312,0.000246,0.000246,0.001368,0.002528,0.001441,0.001990,0.046540,0.288249,0.013415
4,0.0,2020,0.262213,0.265727,0.274069,0.279353,0.289844,0.299846,0.313645,0.322321,0.335674,0.336675,0.336343,0.343257,0.351331,0.341621,0.331728,0.327081,-0.000209,0.000219,0.002457,0.002244,0.001561,0.000773,0.002791,-0.000049,0.002093,-0.000363,0.000154,0.001423,0.000789,-0.003160,-0.000687,-0.000606,0.002619,0.001189,0.005634,0.004793,0.003386,0.001626,0.005602,0.001532,0.004388,0.000928,0.000499,0.002845,0.001633,0.006496,0.001374,0.001213,0.093452,0.313878,0.029333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32377,3597.0,2020,0.331854,0.312785,0.295771,0.293095,0.297096,0.306733,0.315427,0.317786,0.304524,0.297996,0.298956,0.294725,0.289792,0.277141,0.266841,0.258209,-0.001290,-0.002156,-0.002558,0.000245,0.001216,0.001386,0.000656,0.000491,-0.002895,0.000078,0.000061,-0.000952,0.000092,-0.002810,-0.000950,-0.001942,0.003243,0.004542,0.005192,0.000776,0.002458,0.002780,0.001380,0.001849,0.005980

In [9]:
df = df.merge(veg_agg, how = 'inner', on = 'plot_id')

In [10]:
df

,plot_id,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_rad,slope_grad,slope_x,slope_y,year,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_cov,ndvi_mean,ndvi_std
0,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076,2016,0.383043,0.373238,0.356197,0.348879,0.346145,0.345504,0.355702,0.381068,0.413351,0.438675,0.443236,0.440521,0.432348,0.424176,0.416003,0.407830,-0.001783,-0.001783,-0.001851,-0.000534,-0.000351,0.000406,0.003347,0.003689,0.005842,0.000652,0.000652,-0.001168,-0.001168,-0.001168,-0.001168,-0.001168,0.001993,0.003565,0.003942,0.001107,0.000702,0.000845,0.006752,0.007378,0.011685,0.001303,0.001303,0.002335,0.002335,0.002335,0.002335,0.002335,0.090644,0.393639,0.035681
1,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076,2017,0.329507,0.322669,0.320248,0.318124,0.322323,0.324225,0.321754,0.322187,0.327781,0.330168,0.331366,0.333434,0.339194,0.339164,0.338132,0.337247,-0.001214,-0.000664,-0.000828,0.000356,0.000338,0.000090,-0.000614,0.001560,0.000333,0.000323,0.000109,0.000761,0.000437,-0.000147,-0.000147,-0.000147,0.000991,0.002033,0.001665,0.000953,0.000702,0.000415,0.001264,0.003149,0.000665,0.000660,0.000218,0.001577,0.001062,0.000295,0.000295,0.000209,0.022024,0.328595,0.007237
2,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076,2018,0.313716,0.306276,0.305823,0.303524,0.306111,0.314969,0.329831,0.339823,0.344996,0.351547,0.354465,0.366220,0.378805,0.389178,0.428417,0.467546,-0.000188,-0.001330,-0.000526,0.000361,0.000039,0.002017,0.002040,0.000333,0.001936,0.000417,0.000417,0.003317,0.001256,0.002308,0.008811,0.001145,0.000466,0.003144,0.001261,0.000954,0.000679,0.004080,0.004080,0.000839,0.004117,0.000834,0.000834,0.006976,0.002513,0.004907,0.017655,0.002290,0.170850,0.365089,0.062376
3,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701

### Now let's add some weather information

First load the wather that has been unzipped and clipped to the vineyard

In [11]:
weather = pd.read_pickle('../data/PRISM/df.pkl')



weather = (
    weather
    .groupby("date", as_index=False)
    .first()
)


weather['date'] = pd.to_datetime(weather['date'])
weather['doy'] = weather['date'].dt.dayofyear
weather['year'] = weather['date'].dt.year

weather = weather.drop(weather[weather['year'] == 2025].index)
weather

,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year
0,2016-01-01,1,0.0000,-0.415000,-3.15300,-5.8920,2.1920,0.4860,1,2016
1,2016-01-02,1,0.0000,-4.138000,-5.38000,-6.6220,1.1680,0.5970,2,2016
2,2016-01-03,1,0.0000,-4.943000,-6.42400,-7.9050,1.1660,0.5900,3,2016
3,2016-01-04,1,1.0160,-4.861000,-6.44800,-8.0370,0.8410,0.2900,4,2016
4,2016-01-05,1,0.0000,-2.711000,-4.31500,-5.9210,1.1300,0.2980,5,2016
...,...,...,...,...,...,...,...,...,...,...
3283,2024-12-27,1,5.5226,12.897300,8.57640,4.2555,5.3272,0.0464,362,2024
3284,2024-12-28,1,2.7350,9.500899,6.92395,4.3470,1.8989,0.0603,363,2024
3285,2024-12-29,1,6.4769,13.031700,9.00255,4.9734,4.6196,0.0594,364,2024
3286,2024-12-30,1,0.9107,9.572500,6.35390,3.1353,2.6949,0.1705,365,2024


### Crack at some frost and growing degree days cumulative per month

In [12]:
# Compute frost days and GDD
weather['frost'] = (weather['tmin'] < 0)
weather['gdd'] = (weather['tmean'] - 10).clip(lower=0) 

# disregard 2025 for now
weather = weather[weather['year'] != 2025].copy()

In [13]:
# Compute cumulative GDD for each month
weather['week'] = weather['date'].dt.week
cumulative_gdd = weather.groupby([ 'year', 'week'])['gdd'].sum().groupby(level=[0,1]).cumsum()
# Reset index to turn MultiIndex into columns
cumulative_gdd = cumulative_gdd.rename('cumulative_gdd').reset_index()

# Merge back to weather
weather = weather.merge(
    cumulative_gdd,
    on=['year', 'week'],
    how='left'
)


weather

/home/simonhans/anaconda3/envs/GrapeExpectations/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year,frost,gdd,week,cumulative_gdd
0,2016-01-01,1,0.0000,-0.415000,-3.15300,-5.8920,2.1920,0.4860,1,2016,True,0.0,53,0.0
1,2016-01-02,1,0.0000,-4.138000,-5.38000,-6.6220,1.1680,0.5970,2,2016,True,0.0,53,0.0
2,2016-01-03,1,0.0000,-4.943000,-6.42400,-7.9050,1.1660,0.5900,3,2016,True,0.0,53,0.0
3,2016-01-04,1,1.0160,-4.861000,-6.44800,-8.0370,0.8410,0.2900,4,2016,True,0.0,1,0.0
4,2016-01-05,1,0.0000,-2.711000,-4.31500,-5.9210,1.1300,0.2980,5,2016,True,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2024-12-27,1,5.5226,12.897300,8.57640,4.2555,5.3272,0.0464,362,2024,False,0.0,52,0.0
3284,2024-12-28,1,2.7350,9.500899,6.92395,4.3470,1.8989,0.0603,363,2024,False,0.0,52,0.0
3285,2024-12-29,1,6.4769,13.031700,9.00255,4.9734,4.6196,0.0594,364,2024,False,0.0,52,0.0
3286,2024-12-30,1,0.9107,9.572500,6.35390,3.1353,2.6949,0.1705,365,2024,False,0.0,1,0.0


In [14]:
weather = weather[
    (weather['week'] > 27) &
    (weather['week'] < 44)
].copy()

In [15]:
# Define which aggregations to use
agg_funcs = {
    'ppt': 'sum',
    'tmax': 'max',
    'tmin': 'min',
    'tmean': 'mean',
    'vpdmax': 'max',
    'vpdmin': 'min',
    'cumulative_gdd': 'max'
}

# Aggregate to long-form first (year, week, variables)
weekly_long = weather.groupby(['year','week']).agg(agg_funcs).reset_index()

# Pivot to wide form (one row per year, columns per week)
weekly_wide = pd.DataFrame({'year': weekly_long['year'].unique()})

for col in ['ppt','tmax','tmin','tmean','vpdmax','vpdmin','cumulative_gdd']:
    pivoted = weekly_long.pivot(index='year', columns='week', values=col)
    # Rename columns to include variable name
    pivoted.columns = [f'{col}_{w}' for w in pivoted.columns]
    weekly_wide = weekly_wide.merge(pivoted, on='year', how='left')

weekly_wide.reset_index(drop=True, inplace=True)

In [16]:
weekly_wide

,year,ppt_28,ppt_29,ppt_30,ppt_31,ppt_32,ppt_33,ppt_34,ppt_35,ppt_36,ppt_37,ppt_38,ppt_39,ppt_40,ppt_41,ppt_42,ppt_43,tmax_28,tmax_29,tmax_30,tmax_31,tmax_32,tmax_33,tmax_34,tmax_35,tmax_36,tmax_37,tmax_38,tmax_39,tmax_40,tmax_41,tmax_42,tmax_43,tmin_28,tmin_29,tmin_30,tmin_31,tmin_32,tmin_33,tmin_34,tmin_35,tmin_36,tmin_37,tmin_38,tmin_39,tmin_40,tmin_41,tmin_42,tmin_43,tmean_28,tmean_29,tmean_30,tmean_31,tmean_32,tmean_33,tmean_34,tmean_35,tmean_36,tmean_37,tmean_38,tmean_39,tmean_40,tmean_41,tmean_42,tmean_43,vpdmax_28,vpdmax_29,vpdmax_30,vpdmax_31,vpdmax_32,vpdmax_33,vpdmax_34,vpdmax_35,vpdmax_36,vpdmax_37,vpdmax_38,vpdmax_39,vpdmax_40,vpdmax_41,vpdmax_42,vpdmax_43,vpdmin_28,vpdmin_29,vpdmin_30,vpdmin_31,vpdmin_32,vpdmin_33,vpdmin_34,vpdmin_35,vpdmin_36,vpdmin_37,vpdmin_38,vpdmin_39,vpdmin_40,vpdmin_41,vpdmin_42,vpdmin_43,cumulative_gdd_28,cumulative_gdd_29,cumulative_gdd_30,cumulative_gdd_31,cumulative_gdd_32,cumulative_gdd_33,cumulative_gdd_34,cumulative_gdd_35,cumulative_gdd_36,cumulative_gdd_37,cumulative_gdd_38,cumulative_gdd_39,cumulative_gdd_40,cumulative_gdd_41,cumulative_gdd_42,cumulative_gdd_43
0,2016,0.416,1.3400,0.000,0.0,1.6360,0.0000,0.0000,0.0000,4.3250,1.3320,0.0000,0.0000,2.0330,18.859,5.3610,13.6930,33.933998,36.145000,38.367001,35.361000,36.619999,36.757000,34.264999,32.931000,32.688999,31.804001,24.833000,29.313000,24.534000,20.968000,18.017000,17.618000,13.613000,13.3720,13.302000,12.517000,13.460000,13.329000,11.484000,9.587000,8.853000,5.9150,6.3970,7.0000,6.6190,1.4650,4.2890,5.2750,21.465430,22.332858,26.459287,22.403715,22.534715,25.704573,22.136715,20.529573,18.433572,18.298287,15.403572,18.090858,14.384429,11.806572,11.707715,10.577429,47.518002,50.323002,54.015999,47.854000,51.394001,51.263000,43.153000,39.596001,39.025002,39.612999,18.514000,28.528999,16.724001,16.489000,11.248000,8.908000,2.787000,2.2760,2.110000,2.4050,1.523000,3.2820,2.1670,1.419000,0.6290,1.3100,0.9430,1.337000,0.431000,0.2000,0.2480,0.0680,80.258007,86.330009,115.215010,86.826008,87.743006,109.932009,84.957008,73.707010,59.035007,58.088006,37.825005,56.636005,30.691004,15.366003,11.954004,6.667002
1,2017,0.000,0.0000,0.000,0.0,0.0000,2.5440,0.0000,0.0000,0.0000,0.0000,4.7070,0.0000,0.0000,0.094,24.5000,0.0000,34.581001,34.411999,38.254002,38.081001,38.814999,33.997002,32.930000,35.689999,34.360001,33.986000,24.236000,28.160000,23.789000,19.332001,21.513000,21.073999,13.960000,11.0010,14.709000,15.956000,16.559999,11.318000,10.597000,12.072000,14.824000,6.4470,4.2890,6.9650,3.2310,-0.0070,1.2510,2.8320,24.611287,22.399144,25.903144,27.000430,27.344001,21.954144,21.872573,24.333573,23.406716,18.860001,13.780715,16.853001,13.026286,9.473572,11.262429,11.140000,47.055000,45.317001,54.915001,57.021000,54.651001,41.394001,38.148998,46.062000,40.373001,42.230000,22.233999,26.292999,22.930000,15.981000,16.167999,13.167000,4.601000,1.9010,4.728000,3.9800,3.357000,1.6780,1.8130,2.348000,1.8210,1.4510,0.2010,0.500000,0.598000,0.2080,0.1370,0.0770,102.279007,86.794006,111.322010,119.003012,121.408010,83.679008,83.108009,100.335009,93.847010,62.020006,26.465004,47.971006,21.184004,2.869002,9.743004,8.539002
2,2018,0.000,0.0000,0.000,0.0,0.0000,0.0000,0.0000,0.5440,0.0000,0.0000,0.0000,0.0000,8.4310,8.891,0.0000,1.1860,39.570000,38.115002,37.171001,39.797001,39.551998,36.111000,32.799000,32.681000,33.730999,28.233000,24.861000,27.923000,23.375999,21.621000,18.730000,19.798000,13.053000,12.1540,13.477000,12.524000,14.212000,12.941000,11.071000,9.873000,10.033000,7.6370,6.3230,4.9720,3.3190,3.1300,1.3350,3.3060,24.537287,24.634144,26.210001,25.791716,26.438001,23.559430,21.511572,19.734716,20.576001,17.398287,15.469429,15.941001,12.131286,11.408858,10.153000,12.181572,66.167999,57.990002,53.651001,64.530998,59.410000,45.728001,36.759998,41.112000,44.292000,30.368000,22.909000,29.009001,18.936001,17.684999,14.729000,10.745000,2.149000,2.8330,2.944000,1.3600,2.669000,1.6180,1.8190,0.729000,1.4440,0.8080,0.5710,0.557000,0.238000,0.0640,0.16

### Now we can combine data.

In [17]:
df['year'] = df['year'].astype(int)
df = pd.merge(df, weekly_wide, how = 'inner', on = 'year')

In [18]:
df

,plot_id,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_rad,slope_grad,slope_x,slope_y,year,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_cov,ndvi_mean,ndvi_std,ppt_28,ppt_29,ppt_30,ppt_31,ppt_32,ppt_33,ppt_34,ppt_35,ppt_36,ppt_37,ppt_38,ppt_39,ppt_40,ppt_41,ppt_42,ppt_43,tmax_28,tmax_29,tmax_30,tmax_31,tmax_32,tmax_33,tmax_34,tmax_35,tmax_36,tmax_37,tmax_38,tmax_39,tmax_40,tmax_41,tmax_42,tmax_43,tmin_28,tmin_29,tmin_30,tmin_31,tmin_32,tmin_33,tmin_34,tmin_35,tmin_36,tmin_37,tmin_38,tmin_39,tmin_40,tmin_41,tmin_42,tmin_43,tmean_28,tmean_29,tmean_30,tmean_31,tmean_32,tmean_33,tmean_34,tmean_35,tmean_36,tmean_37,tmean_38,tmean_39,tmean_40,tmean_41,tmean_42,tmean_43,vpdmax_28,vpdmax_29,vpdmax_30,vpdmax_31,vpdmax_32,vpdmax_33,vpdmax_34,vpdmax_35,vpdmax_36,vpdmax_37,vpdmax_38,vpdmax_39,vpdmax_40,vpdmax_41,vpdmax_42,vpdmax_43,vpdmin_28,vpdmin_29,vpdmin_30,vpdmin_31,vpdmin_32,vpdmin_33,vpdmin_34,vpdmin_35,vpdmin_36,vpdmin_37,vpdmin_38,vpdmin_39,vpdmin_40,vpdmin_41,vpdmin_42,vpdmin_43,cumulative_gdd_28,cumulative_gdd_29,cumulative_gdd_30,cumulative_gdd_31,cumulative_gdd_32,cumulative_gdd_33,cumulative_gdd_34,cumulative_gdd_35,cumulative_gdd_36,cumulative_gdd_37,cumulative_gdd_38,cumulative_gdd_39,cumulative_gdd_40,cumulative_gdd_41,cumulative_gdd_42,cumulative_gdd_43
0,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,"POLYGON ((-119.78208 45.87245, -119.78215 45.8...",1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076,2016,0.383043,0.373238,0.356197,0.348879,0.346145,0.345504,0.355702,0.381068,0.413351,0.438675,0.443236,0.440521,0.432348,0.424176,0.416003,0.407830,-0.001783,-0.001783,-0.001851,-0.000534,-0.000351,0.000406,0.003347,0.003689,0.005842,0.000652,0.000652,-0.001168,-0.001168,-0.001168,-0.001168,-0.001168,0.001993,0.003565,0.003942,0.001107,0.000702,0.000845,0.006752,0.007378,0.011685,0.001303,0.001303,0.002335,0.002335,0.002335,0.002335,0.002335,0.090644,0.393639,0.035681,0.416,1.3400,0.0,0.0,1.636,0.0,0.0,0.0,4.3250,1.3320,0.0,0.000,2.033,18.859,5.3610,13.6930,33.933998,36.145000,38.367001,35.361000,36.619999,36.757000,34.264999,32.931000,32.688999,31.804001,24.833000,29.313,24.534,20.968000,18.0170,17.618000,13.613000,13.3720,13.302000,12.5170,13.4600,13.3290,11.484000,9.5870,8.8530,5.915,6.397,7.0000,6.6190,1.4650,4.2890,5.2750,21.465430,22.332858,26.459287,22.403715,22.534715,25.704573,22.136715,20.529573,18.433572,18.298287,15.403572,18.090858,14.384429,11.806572,11.707715,10.577429,47.518002,50.323002,54.015999,47.854000,51.394001,51.263000,43.153000,39.596001,39.025002,39.612999,18.514000,28.528999,16.724001,16.489,11.248000,8.908000,2.7870,2.2760,2.1100,2.4050,1.5230,3.282

In [19]:
# # Optional: could also add slope magnitude transforms
# df['slope_squared'] = df['slope_mean'] ** 2
# df['slope_log'] = np.log1p(df['slope_mean'])  # log(1 + slope)

for i in range(28, 44, 1):
    
    df[f'water_availability_{i}'] = df[f'ppt_{i}'] / (1 +  df[f'cumulative_gdd_{i}'])
    df[f'diurnal_temp_range_{i}'] = df[f'tmax_{i}'] / df[f'tmin_{i}']
    df[f'stress_index{i}'] = df[f'vpdmax_{i}'] / (df[f'ppt_{i}'] + 0.1)
df['local_relief'] = df['elev_mean'] - df['elev_min']

df['total_relief_log'] = np.log1p(df['total_relief'])

In [20]:
soil = pd.read_pickle('../data/soil/plot_summary.pkl')

In [21]:
df = pd.merge(df, soil, how = 'inner', on = 'plot_id')

In [22]:
df = pd.DataFrame(df.drop(columns = 'geometry'))

In [24]:
df

,plot_id,curve_mean,curve_min,curve_max,pro_curve_mean,pro_curve_min,pro_curve_max,plan_curve_mean,plan_curve_min,plan_curve_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,total_relief,area_m2,area_ha,aspect_min_cos,aspect_min_sin,aspect_max_cos,aspect_max_sin,aspect_mean_cos,aspect_mean_sin,slope_rad,slope_grad,slope_x,slope_y,year,ndvi_smooth_mean_28,ndvi_smooth_mean_29,ndvi_smooth_mean_30,ndvi_smooth_mean_31,ndvi_smooth_mean_32,ndvi_smooth_mean_33,ndvi_smooth_mean_34,ndvi_smooth_mean_35,ndvi_smooth_mean_36,ndvi_smooth_mean_37,ndvi_smooth_mean_38,ndvi_smooth_mean_39,ndvi_smooth_mean_40,ndvi_smooth_mean_41,ndvi_smooth_mean_42,ndvi_smooth_mean_43,ndvi_smooth_slope_28,ndvi_smooth_slope_29,ndvi_smooth_slope_30,ndvi_smooth_slope_31,ndvi_smooth_slope_32,ndvi_smooth_slope_33,ndvi_smooth_slope_34,ndvi_smooth_slope_35,ndvi_smooth_slope_36,ndvi_smooth_slope_37,ndvi_smooth_slope_38,ndvi_smooth_slope_39,ndvi_smooth_slope_40,ndvi_smooth_slope_41,ndvi_smooth_slope_42,ndvi_smooth_slope_43,ndvi_smooth_std_28,ndvi_smooth_std_29,ndvi_smooth_std_30,ndvi_smooth_std_31,ndvi_smooth_std_32,ndvi_smooth_std_33,ndvi_smooth_std_34,ndvi_smooth_std_35,ndvi_smooth_std_36,ndvi_smooth_std_37,ndvi_smooth_std_38,ndvi_smooth_std_39,ndvi_smooth_std_40,ndvi_smooth_std_41,ndvi_smooth_std_42,ndvi_smooth_std_43,ndvi_cov,ndvi_mean,ndvi_std,ppt_28,ppt_29,ppt_30,ppt_31,ppt_32,ppt_33,ppt_34,ppt_35,ppt_36,ppt_37,ppt_38,ppt_39,ppt_40,ppt_41,ppt_42,ppt_43,tmax_28,tmax_29,tmax_30,tmax_31,tmax_32,tmax_33,tmax_34,tmax_35,tmax_36,tmax_37,tmax_38,tmax_39,tmax_40,tmax_41,tmax_42,tmax_43,tmin_28,tmin_29,tmin_30,tmin_31,tmin_32,tmin_33,tmin_34,tmin_35,tmin_36,tmin_37,tmin_38,tmin_39,tmin_40,tmin_41,tmin_42,tmin_43,tmean_28,tmean_29,tmean_30,tmean_31,tmean_32,tmean_33,tmean_34,tmean_35,tmean_36,tmean_37,tmean_38,tmean_39,tmean_40,tmean_41,tmean_42,tmean_43,vpdmax_28,vpdmax_29,vpdmax_30,vpdmax_31,vpdmax_32,vpdmax_33,vpdmax_34,vpdmax_35,vpdmax_36,vpdmax_37,vpdmax_38,vpdmax_39,vpdmax_40,vpdmax_41,vpdmax_42,vpdmax_43,vpdmin_28,vpdmin_29,vpdmin_30,vpdmin_31,vpdmin_32,vpdmin_33,vpdmin_34,vpdmin_35,vpdmin_36,vpdmin_37,vpdmin_38,vpdmin_39,vpdmin_40,vpdmin_41,vpdmin_42,vpdmin_43,cumulative_gdd_28,cumulative_gdd_29,cumulative_gdd_30,cumulative_gdd_31,cumulative_gdd_32,cumulative_gdd_33,cumulative_gdd_34,cumulative_gdd_35,cumulative_gdd_36,cumulative_gdd_37,cumulative_gdd_38,cumulative_gdd_39,cumulative_gdd_40,cumulative_gdd_41,cumulative_gdd_42,cumulative_gdd_43,water_availability_28,diurnal_temp_range_28,stress_index28,water_availability_29,diurnal_temp_range_29,stress_index29,water_availability_30,diurnal_temp_range_30,stress_index30,water_availability_31,diurnal_temp_range_31,stress_index31,water_availability_32,diurnal_temp_range_32,stress_index32,water_availability_33,diurnal_temp_range_33,stress_index33,water_availability_34,diurnal_temp_range_34,stress_index34,water_availability_35,diurnal_temp_range_35,stress_index35,water_availability_36,diurnal_temp_range_36,stress_index36,water_availability_37,diurnal_temp_range_37,stress_index37,water_availability_38,diurnal_temp_range_38,stress_index38,water_availability_39,diurnal_temp_range_39,stress_index39,water_availability_40,diurnal_temp_range_40,stress_index40,water_availability_41,diurnal_temp_range_41,stress_index41,water_availability_42,diurnal_temp_range_42,stress_index42,water_availability_43,diurnal_temp_range_43,stress_index43,local_relief,total_relief_log,sandtotal_r,silttotal_r,claytotal_r,awc_r,cec7_r,om_r,ph1to1h2o_r,ec_r,profile_depth,max_depth,frag3to10_r,fraggt10_r,dbovendry_r,caco3_r
0,0,0.000489,-0.009361,0.013225,0.000620,-0.010633,0.013191,0.000489,-0.009361,0.013225,207.326294,208.851974,208.022307,22.362457,23.888138,23.058465,1.525681,451.141838,0.045114,-0.909044,0.416701,-0.983593,-0.180402,-0.996750,0.080559,0.087617,0.087841,-0.087556,0.007076,2016,0.383043,0.373238,0.356197,0.348879,0.346145,0.345504,0.355702,0.381068,0.413351,0.438675,0.443236,0.440521,0.432348,0.424176,0.416003,0.407

In [23]:
df.to_pickle('../data/df.pkl')